## Baseline Model using TPOT

In [1]:
import os
import pandas as pd

In [3]:
df = pd.read_csv('../../data/raw/training.csv', index_col=0)
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,...,Q17,Q18,Q19,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification
ID,,,,,,,,,,,,,,,,,,,,,
5086,98,2,3,1,1,2,2,0,0,0,...,-1,4,4,-4.460442,29.811396,0,0,0,0,0
1258,40,1,1,3,5,1,1,1,0,0,...,4,1,4,-6.176438,39.244871,1,1,1,0,3
331,18,2,4,6,3,2,1,0,0,0,...,-1,1,1,-6.825702,37.652798,1,0,0,0,2
6729,50,1,1,3,1,1,1,0,0,0,...,-1,1,4,-3.372049,35.808307,1,0,1,0,3
8671,34,1,1,1,1,2,1,0,1,0,...,-1,1,4,-7.179645,31.039095,1,1,0,1,3


## Create labels

1. No_financial_services: Individuals who do not use mobile money, do not save, do not have credit, and do not have insurance
2. Other_only: Individuals who do not use mobile money, but do use at least one of the other financial services (savings, credit, insurance)
3. Mm_only: Individuals who use mobile money only
4. Mm_plus: Individuals who use mobile money and also use at least one of the other financial services (savings, credit, insurance)

In [ ]:
no_fs = 'mobile_money == 0 and savings == 0 and borrowing == 0 and insurance == 0'
other_only = 'mobile_money == 0 and (savings == 1 or borrowing == 1 or insurance == 1)'
mm_only = 'mobile_money == 1 and (borrowing == 0 and insurance == 0 and borrowing ==0)'
mm_plus = 'mobile_money == 1 and (savings == 1 or borrowing == 1 or insurance == 1)'

In [7]:
X = df.drop(['mobile_money','savings','borrowing','insurance','mobile_money_classification'], axis=1)
y = df['mobile_money_classification']

In [19]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(
    generations=20, 
    population_size=20, 
    verbosity=2,
    n_jobs=-1, 
)
tpot.fit(X, y)

Generation 1 - Current best internal CV score: 0.6664701067344536
Generation 2 - Current best internal CV score: 0.6664701067344536
Generation 3 - Current best internal CV score: 0.6692908823701424
Generation 4 - Current best internal CV score: 0.6702795787302108
Generation 5 - Current best internal CV score: 0.6702795787302108
Generation 6 - Current best internal CV score: 0.6702795787302108
Generation 7 - Current best internal CV score: 0.6702795787302108
Generation 8 - Current best internal CV score: 0.6702795787302108
Generation 9 - Current best internal CV score: 0.6702795787302108
Generation 10 - Current best internal CV score: 0.6702795787302108
Generation 11 - Current best internal CV score: 0.6702795787302108
Generation 12 - Current best internal CV score: 0.6702795787302108
Generation 13 - Current best internal CV score: 0.6702795787302108
Generation 14 - Current best internal CV score: 0.6702795787302108
Generation 15 - Current best internal CV score: 0.6702795787302108
Gene

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
        disable_update_check=False, early_stop=None, generations=20,
        max_eval_time_mins=5, max_time_mins=None, memory=None,
        mutation_rate=0.9, n_jobs=-1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=20,
        random_state=None, scoring=None, subsample=1.0,
        template='RandomTree', use_dask=False, verbosity=2,
        warm_start=False)

In [20]:
test_data = pd.read_csv('../../data/raw/test.csv', index_col=0)

probs = tpot.predict_proba(test_data)

In [21]:
test_sub = pd.read_csv('../../data/raw/sample_submission.csv', index_col=0)
test_sub.head()

,no_financial_services,other_only,mm_only,mm_plus
ID,,,,
2352,0.5423,0.9987,0.12,0.0123
8208,NaN,NaN,NaN,NaN
2785,NaN,NaN,NaN,NaN
2967,NaN,NaN,NaN,NaN
1697,NaN,NaN,NaN,NaN


In [22]:
probs_df = pd.DataFrame(probs, columns=list(test_sub))
probs_df.index = test_sub.index.values

probs_df.head()

,no_financial_services,other_only,mm_only,mm_plus
2352,0.000023,0.000633,0.202193,0.797151
8208,0.024557,0.077552,0.156963,0.740927
2785,0.000938,0.028037,0.130534,0.840491
2967,0.350157,0.542878,0.031094,0.075871
1697,0.000061,0.001230,0.089872,0.908836


In [23]:
probs_df.to_csv('../../data/submissions/baseline.csv')

In [24]:
tpot.export('tpot_baseline_2.py')